In [27]:
__author__ = 'pratyush'
import requests
import json
import os
import modules.config as config
from time import sleep
from requests.auth import HTTPBasicAuth
from ibm_watson import DiscoveryV1
from pprint import pprint
import modules.discovery_helper as dh_help
import modules.common_helper as cm_help
import sys
import pandas as pd
import modules.config as config
import math
import re
import hashlib
import unicodedata
from bs4 import BeautifulSoup 
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
import pickle

In [28]:
all_params = cm_help.get_params()

In [29]:
def discovery_init(COLLECTION_NAME=config.DISCOVERY_COLLECTION_NAME,default=all_params['discovery']['default']):
    response = {}
    
    discovery_param = all_params['discovery']
    # default = 
    authenticator = IAMAuthenticator(apikey=discovery_param[default]['apikey'])
    endurl = "https://gateway.watsonplatform.net/discovery/api"
    discovery = DiscoveryV1(
        version=discovery_param[default]['version'],
        authenticator = authenticator)
    iam_apikey=discovery_param[default]['apikey'],
    url=discovery_param[default]['url']    
    if default =="r3":
        discovery.set_service_url(discovery_param[default]['url'])

    environments = discovery.list_environments().get_result()
    # print(json.dumps(environments, indent=2))
    print("Discovery instance loaded "+default)
    js = json.dumps(environments)
    jres = json.loads(js)
    # print(jres['environments'][1]['environment_id'])
    env_id = jres['environments'][1]['environment_id']
    collections = discovery.list_collections(env_id).get_result()
    cols = json.dumps(collections, indent=2)
    colres = json.loads(cols)

    #print(colres['collections'])
    for item in colres['collections']:
        if item['name']== COLLECTION_NAME:
            print('COLLECTION ID:', item['collection_id'], 'COLLECTION NAME:', item['name'])
            col_id = item['collection_id']
    response['env_id'] = env_id
    response['col_id'] = col_id
    response['discovery'] = discovery
    return response

In [30]:
discovery_config = discovery_init(COLLECTION_NAME='Naviga1',default="r3")

Discovery instance loaded r3
COLLECTION ID: ec586f27-f848-40b7-99af-dab80a43b248 COLLECTION NAME: Naviga1


In [31]:
doc_ids = ['167424f8-ca7f-4224-a00b-a54a5a7766c7',
 '7185ebf3-fa27-4fdc-a9fd-2011165a8603',
 '16601478-ebd3-4b55-bd54-e292865696e1',
 'a6f63029-6c69-49ea-a0dc-a5c7e85f55fb',
 '70ebe48c-a75d-4236-9ed6-4af0d8e1a421',
 '172ac60b-3611-4684-9c75-612abe362a84',
 '92123e9a-a03a-48f1-9f57-59319648b9a0',
 'a1433e93-fa9b-4f17-baaa-c64750ef109d',
 'e1410f9c-99a1-473c-83c3-fccb3b3b2ab2',
 'af8903b0-7658-4461-a770-960dbab328f0',
 'ca53af4f-991f-4712-81e0-ddd1591b1705',
 '79415524-ac65-4cee-99b7-18fc7265a0be',
 'ddb781fd-762a-4a61-9969-cac261425f63',
 'bda70352-5984-46a9-a891-119037c9c63d',
 '2ae29870-e433-4757-a94d-c495690238f5',
 'ff516124-45e4-460d-b6d9-48fad5f25c62',
 '2f872a57-25f2-45ea-ab4c-f02bf10c904c',
 '5eceab7f-f37e-44f4-aba1-19ad9b10f8da',
 'd5d89205-adb4-4139-8bf9-e958674a0da7',
 '0d06446d-a8fb-422d-a8a2-8e69c9be04ff',
 'f63cea4c-f478-4874-a350-41ec049ba56a',
 '58ca5429-675c-442c-98de-ad84fd2cf8c2',
 '60405989-66ba-421d-a062-5848eb7b6dc7',
 '97c847ae-5055-41ca-9f86-ea055963f1e7',
 'c637a047-60ff-4cb3-969e-5a0028b45cea',
 'bd8ad3d7-68ca-4d46-a82b-e1ea096afa26',
 'ddc9b34b-3d42-4153-b618-8d0da636c5c6',
 '6cda90e4-92dc-403b-9515-904ce543b138',
 '5723d949-29e0-4215-ba56-16f64ed33829',
 'e11c8456-3a7b-48c8-909f-1a5993b17091',
 'e3ea4a1c-e43f-4c4b-b491-87bf48adf89f',
 '1e948691-372e-4860-86e9-9f594ae98918',
 '11bab4cb-310f-43b0-8e0c-51dee6d2aa9f',
 '41e96803-31a4-47bf-9e5b-9af18497f0fb',
 '29733af7-66b6-461c-ac93-4cd20ef38ca8',
 'f6f391ca-749a-4bfa-9d9c-193104a5d237',
 'dcb1a853-c0a1-49a8-b2dd-674e06b80a39',
 'd5f19b61-ce53-4609-b802-857614b34537',
 'ff0587d0-5c06-412e-885a-31faf0151e77',
 '9717f3ec-6d4b-434e-82de-8a52552f8931',
 '9cb2e37a-1a40-4928-be23-d74ae391b89f',
 '87ae9d52-43a2-4b7c-9286-8acc7e2dd1fe',
 '0819535b-e9ea-4d53-9100-22435397bef7',
 '45404433-b299-4c0f-a938-08e88b550135',
 '9020f584-79c9-45c5-b8e4-d02b18227c2f',
 'c77ade55-521b-410e-8912-0ec5d0dfaf32',
 '9cc9f69d-ad0d-47ab-a0ef-79fb5584b301',
 '4b803714-d4b3-4420-baec-5b5334f2121d',
 'fb936514-3376-4f45-aabe-1688b1fd569f',
 '21b0f146-d8ea-4a82-9c76-8ffa13513b41']

In [20]:
result = []
offset = 0

count = 1000

for j in range(50):
    reqparams = {'offset':offset,'count':count}
    reqparams['filters']='id::'+sayan_id[j]
    discovery_data2 = dh_help.getDiscoveryData(discovery_config,reqparams)
    
    res={}

    #doc = discovery_data2["results"]
    res["id"]=sayan_id[j]

    source = discovery_data2["results"][0]['html']
    batRegex = re.compile(r'(.<[a-zA-Z]*[^>]*>(.+?)</[a-zA-Z]*>)')

    lists = re.findall(batRegex,source)
    len_ = len(lists)
    for i in range(len_):
        lists[i] = list(lists[i])

    regex = re.compile(r'\svalue="(.*?)"')
    subst = ''
    for i in range(len_):
        bregex = re.findall(regex,lists[i][0])
        if(len(bregex)!=0):
            lists[i][0] = lists[i][0].replace(bregex[0], subst)

    regex = re.compile(r'\sidsrc="(.*?)"')
    for i in range(len_):
        bregex = re.findall(regex,lists[i][0])
        if(len(bregex)!=0):
            lists[i][0] = lists[i][0].replace(bregex[0], subst)
            
    regex = re.compile(r'\shref="(.*?)"')
    for i in range(len_):
        bregex = re.findall(regex,lists[i][0])
        if(len(bregex)!=0):
            lists[i][0] = lists[i][0].replace(bregex[0], subst)
    

    for i in range(len_):
        lists[i][0]=lists[i][0].rsplit('/',1)
        lists[i][0][1]=lists[i][0][1].rsplit('>',1)
        keyword = lists[i][0][1][0]
        value = lists[i][1]

        if keyword in res:
            if value not in res[keyword]:
                res[keyword].append(value)       
        else:
            res[keyword] = [value]

    result.append(res)
    


offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0


In [21]:
result

[{'id': '167424f8-ca7f-4224-a00b-a54a5a7766c7',
  'person': ['Kobe Bryant',
   'LeBron James',
   'Michael Jordan',
   'Bill Russel',
   'Robert Horry',
   'Scottie Pippen',
   'Magic Johnson',
   'Kareem Abdul-Jabbar',
   'Oscar Robertson',
   'Russel Westbrook',
   'Hakeem Olajuwon',
   'Larry Bird',
   'Larry Legend',
   "Shaquille O'Neal",
   'Ervin "Magic" Johnson',
   'Wilt Chamberlain',
   "Wilt Chamberlain's",
   'LeBon James',
   'Kareem Abdul Jabar',
   'John Wooden',
   'Lew Alcindor',
   'Babe Ruth'],
  'org': ["Chicago Bulls'",
   "ESPN's",
   'Boston Celtics',
   'Houston Rockets',
   'Orlando Magic',
   'Miami Heat',
   'UCLA',
   'Chicago Bulls',
   'Washington Wizards'],
  'location': ['Mount Rushmore', 'Los Angeles']},
 {'id': '7185ebf3-fa27-4fdc-a9fd-2011165a8603',
  'location': ['Des Plaines American Legion Post 36',
   'Edward Hotel',
   'Mannheim Road',
   'Rosemont',
   'Park Ridge',
   'Chicago'],
  'chron': ['Aug. 1', 'March 21, 2021', 'Friday, May 29'],
  'per

In [25]:
total_org = 0
for i in range(50):
    if "org" in result[i].keys():
        total_org += len(result[i]["org"])

In [26]:
total_org # Total different org entities extracted by Naviga from 50 documents

199

In [73]:
df = pd.DataFrame(columns=["CompanyCode","Company"])

In [74]:
df

,CompanyCode,Company


In [75]:
result = []
offset = 0

count = 1000

k=0
count_codes = 0

for j in range(50):
    
    df.loc[k,"CompanyCode"] = "Document "+str(j)
    df.loc[k,"Company"] = "Document "+str(j)
    k+=1
    df.loc[k,"CompanyCode"] = ""
    df.loc[k,"Company"] = ""
    k+=1
    
    reqparams = {'offset':offset,'count':count}
    reqparams['filters']='id::'+sayan_id[j]
    discovery_data2 = dh_help.getDiscoveryData(discovery_config,reqparams)
    
    res={}

    #doc = discovery_data2["results"]
    res["id"]=sayan_id[j]

    source = discovery_data2["results"][0]['html']
    batRegex = re.compile(r'(.<[a-zA-Z]*[^>]*>(.+?)</[a-zA-Z]*>)')

    lists = re.findall(batRegex,source)
    len_ = len(lists)
    for i in range(len_):
        lists[i] = list(lists[i])

    regex = re.compile(r'\svalue="(.*?)"')
    subst = ''


    for i in range(len_):
        bregex = re.findall(regex,lists[i][0])
        if(len(bregex)!=0):
            lists[i][0] = lists[i][0].replace(bregex[0], subst)

            lists[i][0]=lists[i][0].rsplit('/',1)
            lists[i][0][1]=lists[i][0][1].rsplit('>',1)
            keyword = lists[i][0][1][0]
            value = lists[i][1]
            
            if keyword == "org":
                if (bregex[0],value) not in zip(df["CompanyCode"],df["Company"]):
                    df.loc[k,"CompanyCode"] = bregex[0]
                    df.loc[k,"Company"] = value
                    k+=1
                    count_codes+=1
        else:
            
            lists[i][0]=lists[i][0].rsplit('/',1)
            lists[i][0][1]=lists[i][0][1].rsplit('>',1)
            keyword = lists[i][0][1][0]
            value = lists[i][1]
            if keyword == "org": 
                if value not in list(df["Company"]):
                    df.loc[k,"CompanyCode"] = ""
                    df.loc[k,"Company"] = value
                    k+=1
    k+=1
    df.loc[k,"CompanyCode"] = ""
    df.loc[k,"Company"] = ""
    k+=1
    


offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0


In [76]:
df.head()

,CompanyCode,Company
0,Document 0,Document 0
1,,
2,ACORN:6001131364,Chicago Bulls'
3,,ESPN's
4,ACORN:6001131381,Boston Celtics


In [77]:
df.shape

(342, 2)

In [79]:
df.to_excel("CompanyCodes.xlsx",index=False)

In [80]:
count_codes  # Number of different CompanyCodes in 50 documents

51

In [85]:
print("")
print("Count_codes : ",count_codes)
print("")
print("Total_org : ",total_org)
print("")


Count_codes :  51

Total_org :  199



# For counting numbers

In [ ]:
D = pd.DataFrame(columns=["companyCode","org_html","company_disc"])

In [ ]:
D

In [ ]:
data_disc = json.load(open("first_50_sayan.json"))

In [ ]:
data_disc

In [ ]:
#     try:
#         df.loc[j,""] = 
    
#     try:
#         D.loc[j,"companyCode"] = len(discovery_data2["results"][0]['companyCode'])       
#     except:
#         D.loc[j,"companyCode"] = 0
        
#     try:        
#         D.loc[j,"org_html"] = len(result[j]["org"])
#     except:        
#         D.loc[j,"org_html"] = 0
    
#     try:        
#         D.loc[j,"company_disc"] = len(data_disc[j]["Company"])
#     except:        
#         D.loc[j,"company_disc"] = 0
    
#     D.loc[j,"companyCode"] = len(discovery_data2["results"][0]['companyCode'])
#     D.loc[j,"org"] = len(result[j]["org"])

In [56]:
D["companyCode"]=D["companyCode"].replace(26,1)

In [57]:
D

,companyCode,org_html,company_disc
0,7,9,1
1,0,2,0
2,0,0,0
3,4,8,0
4,1,1,1
5,0,3,1
6,0,2,0
7,1,7,2
8,0,1,0
9,0,0,0


In [58]:
offset = 0
count = 10

reqparams = {'offset':offset,'count':count}
reqparams['filters']='id::'+sayan_id[44]
discovery_data2 = dh_help.getDiscoveryData(discovery_config,reqparams)    

offset in query = 0


In [59]:
discovery_data2["results"][0]['companyCode']

['ACORN:A.4098318050#6#60#60',
 'ACORN:A.0757421235#6#60#60',
 'NASDAQ-NMS:FB#6#60#60',
 'ACORN:A.4019377134#6#60#60',
 'NYSE:F#6#60#60',
 'ACORN:A.3125986960#6#60#60',
 'NYSE:AHC#6#60#60',
 'ACORN:A.1554191032#6#60#60',
 'ACORN:A.0580537411#6#60#60',
 'ACORN:A.3974939109#6#60#60',
 'ACORN:A.0153829789#6#60#60']

In [60]:
result[44]["org"]

['Fox News',
 'GoFundMe',
 'The View',
 'who',
 'Facebook',
 'Saddleback College',
 'Sam Houston State',
 'Centennial High School',
 'Ford',
 'The Dallas Morning News',
 "World Health Organization's",
 "Center for Disease Control and Prevention's",
 'Royal Caribbean',
 "ABC's",
 'Blue Bell Creameries',
 'Texas Supreme Court',
 'Lew Sterrett Justice Center',
 'GOP',
 "Small Business Administration's",
 'Republican Party',
 'Texas Department of Licensing and Regulation',
 'Morning News',
 'Sola Salon']

In [61]:
data_disc[44]["Company"]

['Facebook', 'Trump']